In [1]:
from torch.distributions import Bernoulli
import pandas as pd
import numpy as np
import os
import torch

In [37]:
number_of_nodes = 3
number_of_upper_nodes = int(number_of_nodes*(number_of_nodes-1)*.5)
batch_size = 32

upper_matrices = Bernoulli(torch.full((batch_size,number_of_upper_nodes),.5)).sample()

In [39]:
upper_matrices.shape

torch.Size([32, 3])

In [50]:
full_matrices = recover_full_matrices(upper_matrices)

In [47]:
def recover_full_matrices(upper_diagonal_tensor):
    assert len(upper_diagonal_tensor.shape) == 2
    number_of_upper_entries = upper_diagonal_tensor.shape[1]
    batch_size = upper_diagonal_tensor.shape[0]
    
    matrix_size = int(.5*(1 + np.sqrt(1+8*number_of_upper_entries)))

    # Create a zero-filled tensor to hold the full matrices
    full_matrices = torch.zeros(batch_size, matrix_size, matrix_size)

    # Get the indices for the upper diagonal part of the matrices
    upper_tri_indices = torch.triu_indices(matrix_size, matrix_size,offset=1)

    # Fill the upper diagonal part of the matrices
    full_matrices[:, upper_tri_indices[0], upper_tri_indices[1]] = upper_diagonal_tensor

    # Transpose and fill the lower diagonal part to make the matrices symmetric
    full_matrices = full_matrices + full_matrices.transpose(1, 2)

    return full_matrices